Imports
============

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from dvn.dvn_fcn import DeepVesselNetFCN
from dvn.solver import Solver
from dvn.data_utils import SyntheticData

torch.set_default_tensor_type('torch.FloatTensor')
# set up default cuda device
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

Firstly we load the data
============


In [2]:
ps=64
pn=100

train_synthetic = SyntheticData(root_path="./data/train/", patch_size=ps, patch_num=pn)
val_synthetic = SyntheticData(root_path="./data/val/", patch_size=ps, patch_num=pn)

print("DONE")

DONE


Visualize part of the data
============

In [3]:
print("Train size: %i" % len(train_synthetic))
print("Validation size: %i" % len(val_synthetic))
print("Img size: ", train_synthetic[0][0].size())
print("Segmentation size: ", train_synthetic[0][1].size())

# num_example_imgs = 3
# plt.figure(figsize=(10, 5 * num_example_imgs))
# for i, (img, target) in enumerate(train_synthetic[:num_example_imgs]):
#     # img
#     plt.subplot(num_example_imgs, 2, i * 2 + 1)
#     plt.imshow(img[1].numpy())
#     plt.axis('off')
#     if i == 0:
#         plt.title("Input image")
    
#     # target
#     plt.subplot(num_example_imgs, 2, i * 2 + 2)
#     plt.imshow(target[1].numpy())
#     plt.axis('off')
#     if i == 0:
#         plt.title("Target image")
# plt.show()

Train size: 110
Validation size: 26
Img size:  torch.Size([1, 64, 64, 64])
Segmentation size:  torch.Size([1, 64, 64, 64])


Design network 
============

In [4]:
train_loader = torch.utils.data.DataLoader(train_synthetic[0:5], batch_size=1, shuffle=True, num_workers=0)
val_loader = torch.utils.data.DataLoader(val_synthetic[111:121], batch_size=1, shuffle=False, num_workers=0)

model = DeepVesselNetFCN()
model.to(device)

solver = Solver(optim_args={"lr": 0.01, "weight_decay": 0.01})
solver.train(model, train_loader, val_loader, log_nth=5, num_epochs=5)

START TRAIN
[Iteration 5/25] TRAIN loss: 0.631
[Epoch 1/5] TRAIN acc/loss: 0.948/0.631
[Iteration 10/25] TRAIN loss: 0.354
[Epoch 2/5] TRAIN acc/loss: 0.985/0.354
[Iteration 15/25] TRAIN loss: 0.351
[Epoch 3/5] TRAIN acc/loss: 0.958/0.351
[Iteration 20/25] TRAIN loss: 0.350
[Epoch 4/5] TRAIN acc/loss: 0.958/0.350
[Iteration 25/25] TRAIN loss: 0.349
[Epoch 5/5] TRAIN acc/loss: 0.958/0.349
FINISH


Train network
============

Test network
============

Visualization of network outputs
============
